#Imports + Installations

In [1]:
# Importing Libraries
# please do go through this python notebook: 
import warnings
warnings.filterwarnings("ignore")

import csv
import pandas as pd # Pandas to create small dataframes 
import datetime # Convert to unix time
import time # Convert to unix time
# If numpy is not installed already : pip3 install numpy
import numpy as np # Do aritmetic operations on arrays
# Matplotlib: used to plot graphs
import matplotlib
import matplotlib.pylab as plt
import seaborn as sns # Plots
from matplotlib import rcParams # Size of plots  
from sklearn.cluster import MiniBatchKMeans, KMeans # Clustering
import math
import pickle
import os
# To install xgboost: pip3 install xgboost
import xgboost as xgb

import warnings
import networkx as nx
import pdb
import pickle
from tqdm.notebook import tqdm
import os
import random
from sklearn.model_selection import train_test_split

import collections

#Loading the data

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
os.listdir()

['.config', 'gdrive', 'sample_data']

In [5]:
os.listdir('gdrive/My Drive/Major Project')

['soc-Epinions1.txt',
 'pos_samples_test_rand.csv',
 'neg_samples.csv',
 'df_train.csv',
 'pos_samples.csv',
 'Major Projectgraph_wave_embeddings',
 'neg_test.csv',
 'neg_train',
 'combined.csv',
 'combined_test.csv',
 'nodesketch_pkl',
 'train_data_nodesketch_concat',
 'test_data_nodesketch_concat',
 'wys_model.h5',
 'wys_file.pkl',
 'wys_embeddings.pkl',
 'test_91.csv',
 'pos_train_91.csv',
 'fea_sample',
 'wys_embeddings_91.pkl',
 'wys_model_91.h5',
 'wiki',
 'shuffled_df.csv',
 '99:1_shuffled_df.csv',
 'epinions_netmf_train91.pkl',
 'epinions_netmf_test91.pkl',
 'epinions_nodesketch91_pkl',
 'epinions_deepwalk_train91.pkl',
 'epinions_deepwalk_test91.pkl',
 'epinions_walklets_train91.pkl',
 'epinions_walklets_test91.pkl',
 'gcn_model.h5',
 'epinions_gcn_train91.pkl',
 'epinions_gcn_test91.pkl',
 'epinions_node2vec_train91.pkl',
 'epinions_node2vec_test91.pkl',
 'graphsage_model.h5',
 'epinions_gsage_train91.pkl',
 'epinions_gsage_test91.pkl',
 'epinions_graphwave_embeddings_91',
 '

In [3]:
data_path = 'gdrive/My Drive/Major Project'

In [4]:
train = pd.read_csv(data_path+'/soc-Epinions1.txt', sep = "\t", header = None, names = ['Source', 'Destination'])
train.head()

,Source,Destination
0,0,4
1,0,5
2,0,7
3,0,8
4,0,9


In [5]:
nodes = sorted(list(set(train['Source'].values) | set (train['Destination'].values)))
print(len(nodes))
ledger = {}
for i, node in enumerate(nodes):
  ledger[node] = i
print(len(ledger))

75879
75879


In [6]:
friends = nx.DiGraph()
for src, dst in tqdm(train.values):
  friends.add_edge(ledger[src], ledger[dst])

  0%|          | 0/508837 [00:00<?, ?it/s]

In [7]:
df = pd.DataFrame({}, {}, columns = ['src', 'dest'])  
src_list = [ledger[src] for src,dest in train.values]
dest_list = [ledger[dest] for src, dest in train.values]
df['src'] = src_list
df['dest'] = dest_list
df['link'] = 1
print(df.head())

   src  dest  link
0    0     4     1
1    0     5     1
2    0     7     1
3    0     8     1
4    0     9     1


In [8]:
df = df.sample(frac=1)

In [9]:
node1 = set(df['src'])
node2 = set(df['dest'])
df_nodes = node1.union(node2)
print(len(df_nodes))

75879


In [10]:
friends.number_of_nodes()

75879

In [11]:
friends.number_of_edges()

508837

In [12]:
for node in friends.nodes():
  if (len(friends.out_edges(node)) + len(friends.in_edges(node))) == 0:
    print(node)

in_out_zero = []
for node in friends.nodes():
  if len(friends.out_edges(node)) == 0 or len(friends.in_edges(node)) == 0:
    in_out_zero.append(node)

len(in_out_zero)

39460

In [13]:
nodelist = sorted(list(ledger.values()))
nodes = sorted(list(friends.nodes()))
print(len(nodelist))
if nodes == nodelist:
  print("true")
else:
  print("false")

75879
true


In [14]:
friends.adj[3]

AtlasView({1: {}, 5: {}, 18: {}, 28: {}, 29: {}, 30: {}, 31: {}, 32: {}, 34: {}, 92: {}, 198: {}, 211: {}, 352: {}, 359: {}, 404: {}, 437: {}, 452: {}, 550: {}, 640: {}, 665: {}, 681: {}, 682: {}, 683: {}, 684: {}, 685: {}, 686: {}, 687: {}, 688: {}, 689: {}, 690: {}, 691: {}, 692: {}, 693: {}, 694: {}, 695: {}, 696: {}, 697: {}, 698: {}, 699: {}, 700: {}, 701: {}})

# Section 1: Data Preparation

1.1 Extracting positive samples from the graph

The procedure followed is:
* Obtain the adjacency matrix
* Traverse to find nodes that are not connected
* Store these pairs of nodes to use as negative samples during training

In [15]:
list(friends.edges())[:5]

[(0, 4), (0, 5), (0, 7), (0, 8), (0, 9)]

In [16]:
def get_neg_samples(nodes, graph):
  """
  Input: 
    nodes: list of all nodes of the graph
    graph: the nx graph
  Output:
    neg_samples: a pd dataframe containing all pairs of disconnected nodes
  
  Computes the adjacency matrix, and returns all disconnected nodes
  """

  edges = set(graph.edges())

  discon_pairs = set([])

  while (len(discon_pairs)<500000):
    try:
      a=ledger[random.randint(1, 75879)]
      b=ledger[random.randint(1, 75879)]
    except:
      continue
    if a!=b and (a,b) not in edges and (b,a) not in edges:
        try:
            if nx.shortest_path_length(graph,source=a,target=b) > 2: 
                discon_pairs.add((a,b))
            else:
                continue  
        except:  
                discon_pairs.add((a,b))              
    else:
        continue

  # for i in tqdm(range(adj_g.shape[0]-1)):
  #   for j in range(i+1, adj_g.shape[1]):
  #     try:
  #       if nx.shortest_path_length(graph, i, j) <= 2:
  #         if adj_g[i, j] == 0:
  #           discon_pairs.append([nodes[i], nodes[j]])
  #     except nx.NetworkXNoPath:
  #       continue

  print(f"Number of disconnected nodes found = {len(discon_pairs)}")

  #converting the list into a dataframe

  neg_samples = pd.DataFrame({'src': [i[0] for i in discon_pairs], 
                              'dest': [i[1] for i in discon_pairs]})  
  neg_samples['link'] = 0

  return neg_samples

In [17]:
%%time
neg_data = get_neg_samples(nodelist, friends)

# estimated running time for this cell = 4 days
# check for possible places for optimization 

Number of disconnected nodes found = 500000
CPU times: user 23.5 s, sys: 171 ms, total: 23.7 s
Wall time: 24.5 s


In [18]:
neg_data.head()

,src,dest,link
0,44542,10139,0
1,45678,28350,0
2,67540,7751,0
3,29611,15544,0
4,44071,62344,0


1.2 Obtain positive samples from the original graph

These positive samples, along with the negative samples obtained above would be used for training the classifiers. The positive samples are simply **pairs of nodes between which a link actually exists in the original graph.**

The procedure followed is: 
* Check if dropping a node pair results in splitting of the graph (increases the number of conn components)
* Check if dropping the pair results in a reduction in the number of nodes
* If both the above constraints are fulfilled, then drop the node pair (edge) - store this node pair in a list of positive samples

In [19]:
# creating a dictionary of number of times a node occurs in the dict
src = list(df['src'])
dest = list(df['dest'])

src_dict = collections.Counter(src)
dest_dict = collections.Counter(dest)

print(len(src_dict), len(dest_dict))

60341 51957


In [20]:
def get_pos_samples(df, g):
  """
  Input:
    df: the original edgelist as a pd dataframe 
    g: the original networkx graph
    assumes: 
      nodes are specified as Source and Destination
      original Graph is a Digraph
      node indexing is corrected
  Output:
    pos_samples: dataframe containing all removable edges (positive samples)
  """
  df_temp = df.copy()
  gr = g.copy(as_view=False)
  node_count = len(gr.nodes())
  print(f"original node count = {node_count}")
  
  removable_links_idx = []
  
  cc = nx.number_weakly_connected_components(gr)
  print(f"Original number of connected components = {cc}")

  for c, i in tqdm(enumerate(df.index.values)):
    src, dest, link = df.loc[i].values
    
    gr.remove_edge(src,dest)

    if(gr.degree(src) == 0):
      gr.remove_node(src)
    if(gr.degree(dest) == 0):
      gr.remove_node(dest)
  
    if len(gr.nodes()) == node_count:
      removable_links_idx.append(i)
      df_temp = df_temp.drop(index=i)
    else:
      gr.add_edge(src,dest)
    
    if c == 60000:
      print(f"removable links index = {len(removable_links_idx)}")
      break
  
  pos_samples = df.loc[removable_links_idx]
  pos_samples['link'] = 1

  nodes_new = set(df_temp['src']).union(set(df_temp['dest']))
  print(f"New number of nodes = {len(nodes_new)}")

  return pos_samples, df_temp

In [21]:
pos_samples, df_train = get_pos_samples(df, friends)
pos_samples_idx = pos_samples.index.values
print(pos_samples.shape)
pos_samples.head()

original node count = 75879
Original number of connected components = 2


0it [00:00, ?it/s]

removable links index = 56224
New number of nodes = 75879
(56224, 3)


,src,dest,link
247569,7397,3497,1
83447,2224,6953,1
397278,24635,1746,1
253252,1864,24384,1
214701,1056,1915,1


In [22]:
print(df_train.shape)
df_train.head()

(452613, 3)


,src,dest,link
329593,14843,53045,1
502348,65366,38273,1
269592,1862,24370,1
288269,11946,50132,1
499295,59808,69735,1


In [23]:
# sanity check
train_nodes = set(df_train['src']).union(set(df_train['dest']))

In [24]:
len(df_nodes.difference(train_nodes))

0

In [25]:
train_graph = nx.from_pandas_edgelist(df_train, source='src', target='dest', create_using=nx.MultiDiGraph())

In [26]:
original_graph = nx.from_pandas_edgelist(df, source="src", target="dest", create_using=nx.MultiDiGraph)

In [27]:
cc_before = nx.number_weakly_connected_components(original_graph)
print(cc_before)

2


In [28]:
cc_after = nx.number_weakly_connected_components(train_graph)

In [29]:
print(cc_after)

292


In [30]:
neg_data.shape

(500000, 3)

In [31]:
neg_test = neg_data.sample(frac = 0.1)
print(neg_test.shape)
neg_test.head()

(50000, 3)


,src,dest,link
196601,24789,33191,0
493186,13360,61167,0
250436,74538,71689,0
498400,52156,60240,0
255074,34652,52103,0


In [32]:
neg_train = neg_data.drop(neg_test.index.values, axis=0)
print(neg_train.shape)
neg_train.head()

(450000, 3)


,src,dest,link
0,44542,10139,0
1,45678,28350,0
2,67540,7751,0
3,29611,15544,0
4,44071,62344,0


In [33]:
combined_tr = pd.concat([df_train, neg_train], axis = 0).sample(frac=1)
print(combined_tr.shape)
combined_tr.head()

(902613, 3)


,src,dest,link
26102,1972,8899,0
188152,9098,17151,0
394436,19771,46179,0
507726,74448,385,1
498414,58263,69516,1


In [34]:
combined_te = pd.concat([pos_samples, neg_test], axis = 0).sample(frac=1)
print(combined_te.shape)
combined_te.head()

(106224, 3)


,src,dest,link
381519,21371,15072,1
312318,10950,60275,0
493058,67682,39756,0
465962,16581,15980,1
482102,32277,23293,1


In [35]:
combined_tr.to_csv(data_path+'/train_91_2.csv')
combined_te.to_csv(data_path+'/test_91_2.csv')

In [36]:
df_train.to_csv(data_path+'/pos_train_91_2.csv')

In [37]:
combined_tr = pd.read_csv(data_path+'/train_91_2.csv', index_col = [0])
print(combined_tr.shape)
combined_tr.head()

(902613, 3)


,src,dest,link
26102,1972,8899,0
188152,9098,17151,0
394436,19771,46179,0
507726,74448,385,1
498414,58263,69516,1


In [38]:
combined_te = pd.read_csv(data_path+'/test_91_2.csv', index_col = [0])
print(combined_te.shape)
combined_te.head()

(106224, 3)


,src,dest,link
381519,21371,15072,1
312318,10950,60275,0
493058,67682,39756,0
465962,16581,15980,1
482102,32277,23293,1


In [39]:
df_train = pd.read_csv(data_path+'/pos_train_91_2.csv', index_col = [0])
print(df_train.shape)
df_train.head()

(452613, 3)


,src,dest,link
329593,14843,53045,1
502348,65366,38273,1
269592,1862,24370,1
288269,11946,50132,1
499295,59808,69735,1


In [ ]:
train_graph = nx.from_pandas_edgelist(df_train, source='src', target='dest', create_using=nx.MultiDiGraph)

In [ ]:
scales = [5, 10, 20, 50]
sample_points = np.linspace(0, 100, 50).astype(np.float32)
degree = 20
G = StellarGraph.from_networkx(train_graph)
generator = GraphWaveGenerator(G, scales=scales, degree=degree)

In [ ]:
emb = generator.flow(
    node_ids = G.nodes(), sample_points = sample_points, batch_size = 1, repeat = False
)

In [ ]:
embeddings = [x.numpy() for x in tqdm(emb)]

  0%|          | 0/75879 [00:00<?, ?it/s]

In [ ]:
filename = data_path + "_embeddings"

In [ ]:
outfile = open(filename, "wb")
pickle.dump(embeddings, outfile)

In [ ]:
print(filename)

gdrive/My Drive/Major Project_embeddings


In [ ]:
file = open(filename, "rb")
gw_emb = pickle.load(file)
file.close()

EOFError: ignored

In [ ]:
print(len(gw_emb))

75879


In [ ]:
emb_dict = {}
for node, emb in zip(G.nodes(), gw_emb):
  emb_dict[node] = emb
print(len(emb_dict))

75879


In [ ]:
# get training embeddings
t_e = []
for i, row in combined.iterrows():
  comb_emb = emb_dict[row['src']] + emb_dict[row['dest']]
  t_e.append(comb_emb)
print(len(t_e))

999165


In [ ]:
t_y = combined['link']
len(t_y)

999165

In [ ]:

import xgboost as xgb
